# Challenge Machine Learning ANAP

Auteur: TMTC

Date: 12 Janvier 2017

Challenge du WS de X sur les maladies chroniques

## 0. Import the environments

In [25]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import random

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np

import os
import re
from collections import defaultdict

import xlrd
import csv

In [35]:
def convert_point(x):
    return re.sub(',', '.', x)

In [21]:
pd.__version__

'0.19.1'

## 1. Load the data

In [52]:
datapath = "../challenge_28_data/"
trainf = "data2.csv"
dataf = datapath + trainf
train = pd.read_csv(dataf,';')
features_names = list(train)[1:]
train.rename(columns={'Finess':'finess'}, inplace=True)
train.drop('Raison sociale', inplace=True, axis=1)
train.dtypes

finess                                                object
Provenance des patients (département)                 object
Domaines d activités                                  object
âge (deux classes >75 ans, <= 75 ans)                 object
Nombre de séjours/séances MCO des patients en ALD      int64
Nombre total de séjours/séances                        int64
annee                                                  int64
cible1                                               float64
dtype: object

## 2. Add more data

In [53]:
traina08 = "hospidiag_opendata_2008.xlsx"
train08a = pd.read_excel(datapath+traina08,sheetname = 3)
#train08a.drop('A1bis', inplace=True, axis=1)
train08a.drop('zone', inplace=True, axis=1) # For now
train08a.fillna('0', inplace=True)
for col in ['A1bis', 'A2bis', 'A3bis', 'A4bis', 'A5bis']:
    train08a[col] = pd.to_numeric(train08a[col].apply(convert_point))
train08a.dtypes

finess     object
A1bis       int64
A2bis     float64
A3bis     float64
A4bis     float64
A5bis     float64
dtype: object

In [ ]:
#concatenate train08a with initial set of data train, only keeping rows already existing in train
#result = pd.concat([train, train08a], axis=1, join_axes=[train.index])

In [57]:
#merge
result = pd.merge(train, train08a, on='finess')
result

,finess,Provenance des patients (département),Domaines d activités,"âge (deux classes >75 ans, <= 75 ans)",Nombre de séjours/séances MCO des patients en ALD,Nombre total de séjours/séances,annee,cible1,A1bis,A2bis,A3bis,A4bis,A5bis
0,280000449,78-Yvelines,D18-Maladies infectieuses (dont VIH),<=75 ans,0,1,2008,0.000000,0,0.2,0.0,0.0,0.0
1,280000449,78-Yvelines,D19-Endocrinologie,<=75 ans,0,3,2008,0.000000,0,0.2,0.0,0.0,0.0
2,280000449,78-Yvelines,D19-Endocrinologie,>75 ans,0,1,2008,0.000000,0,0.2,0.0,0.0,0.0
3,280000449,78-Yvelines,"D23-Toxicologie, Intoxications, Alcool",>75 ans,1,1,2008,0.124902,0,0.2,0.0,0.0,0.0
4,280000449,78-Yvelines,"D26-Activités inter spécialités, suivi thérape...",>75 ans,1,1,2008,0.124092,0,0.2,0.0,0.0,0.0
5,280000449,81-Tarn,D07-Cardio-vasculaire (hors cathétérismes vasc...,<=75 ans,0,1,2008,0.000000,0,0.2,0.0,0.0,0.0
6,280000449,83-Var,D07-Cardio-vasculaire (hors cathétérismes vasc...,<=75 ans,1,1,2008,0.124902,0,0.2,0.0,0.0,0.0
7,280000449,89-Yonne,D16-Hématologie,>75 ans,1,1,2008,0.124902,0,0.2,0.0,0.0,0.0
8,280000449,91-Essonne,D07-Cardio-vasculaire (hors cathétérismes vasc...,<=75 ans,0,1,2008,0.000000,0,0.2,0.0,0.0,0.0
9,280000449,91-Essonne,D07-Cardio-vasculaire (hors cathétérismes vasc...,>75 ans,1,2,2008,0.068991,0,0.2,0.0,0.0,0.0


In [36]:
train08b = pd.read_excel(datapath+traina08,sheetname = 4)
train08b
train08b.drop('A6', inplace=True, axis=1)
train08b.fillna('0', inplace=True)
for col in ['A1', 'A2', 'A3', 'A4', 'A5']:
    train08b[col] = pd.to_numeric(train08b[col].apply(convert_point))
train08b.dtypes

finess     object
zone       object
A1        float64
A2        float64
A3        float64
A4        float64
A5        float64
dtype: object

In [7]:
traina09 = "hospidiag_opendata_2009.xlsx"
train09a = pd.read_excel(datapath+traina09,sheetname = 3)

traina10 = "hospidiag_opendata_2010.xlsx"
train10a = pd.read_excel(datapath+traina10,sheetname = 3)

traina11 = "hospidiag_opendata_2011.xlsx"
train11a = pd.read_excel(datapath+traina11,sheetname = 3)

traina12 = "hospidiag_opendata_2012.xlsx"
train12a = pd.read_excel(datapath+traina12,sheetname = 3)

traina13 = "hospidiag_opendata_2013.xlsx"
train13a = pd.read_excel(datapath+traina13,sheetname = 3)

traina14 = "hospidiag_opendata_2014.xlsx"
train14a = pd.read_excel(datapath+traina14,sheetname = 3)

traina15 = "hospidiag_opendata_2015.xlsx"
train15a = pd.read_excel(datapath+traina15,sheetname = 3)

train09b = pd.read_excel(datapath+traina09,sheetname = 4)

train10b = pd.read_excel(datapath+traina10,sheetname = 4)

train11b = pd.read_excel(datapath+traina11,sheetname = 4)

train12b = pd.read_excel(datapath+traina12,sheetname = 4)

train13b = pd.read_excel(datapath+traina13,sheetname = 4)

train14b = pd.read_excel(datapath+traina14,sheetname = 4)

train15b = pd.read_excel(datapath+traina15,sheetname = 4)

KeyboardInterrupt: 

## 3. Prepare the data

Discard the redundancy

In [37]:
del train['Finess']
#Finess <=> Raison sociale - We choose here to learn as well using the hospitals name 
#(team/equipments,etc...)

#Age to numeric

train['âge (deux classes >75 ans, <= 75 ans)'][train['âge (deux classes >75 ans, <= 75 ans)']=='<=75 ans']=0
train['âge (deux classes >75 ans, <= 75 ans)'][train['âge (deux classes >75 ans, <= 75 ans)']=='>75 ans']=1
train['âge (deux classes >75 ans, <= 75 ans)'] = pd.to_numeric(train['âge (deux classes >75 ans, <= 75 ans)'])

#Raison sociale / Labelencoder

le.fit(train['Raison sociale'])
list(le.classes_)
train['Raison sociale'] = le.transform(train['Raison sociale'])

#pd.get_dummies
train_DA = pd.get_dummies(train['Domaines d activités'])
train_PP = pd.get_dummies(train['Provenance des patients (département)'])
train_new = pd.concat([train,train_DA,train_PP],axis=1)

#Suppression des doublons

del train_new['Domaines d activités']
del train_new['Provenance des patients (département)']
#del train_new['Finess']

#Reorder the column
cols = train_new.columns.tolist()
cols =  cols[5:] + cols[1:5]
cols
train_new=train_new[cols]

In [38]:
train_new.dtypes

cible1                                                                                      float64
D01-Digestif                                                                                  uint8
D02-Orthopédie traumatologie                                                                  uint8
D03-Traumatismes multiples ou complexes graves                                                uint8
D04-Rhumatologie                                                                              uint8
D05-Système nerveux (hors cathétérismes vasculaires diagnostiques et interventionnels)        uint8
D06-Cathétérismes vasculaires diagnostiques et interventionnels                               uint8
D07-Cardio-vasculaire (hors cathétérismes vasculaires diagnostiques et interventionnels)      uint8
D09-Pneumologie                                                                               uint8
D10-ORL, Stomatologie                                                                         uint8


## 4. Preparing the folds

On pourrait utiliser [`sklearn.cross_validation`](http://scikit-learn.org/stable/modules/cross_validation.html#k-fold) pour préparer les folds mais on le fait "à la main" pour cette première fois.

In [5]:
import random
def kfolds(k,N,seed=None):
    random.seed(seed)
    out = [ list() for _ in range(k) ]
    for n in range(N): out[random.randrange(k)].append(n)
    return(out)

K = kfolds(seed=42,k=5,N=len(train_new))
fold1 = sum(K[1:5],[]) # K[1] + K[2] + K[3] + K[4]
#fold1 = K[1]+K[2]+K[3]+K[4]
xtrain = train_new.iloc[fold1][train_new.keys()[1:]]
ytrain = train_new.iloc[fold1][train_new.keys()[0]]

xtest = train_new.iloc[K[0]][train_new.keys()[1:]]
ytest = train_new.iloc[K[0]][train_new.keys()[0]]

## 5. GradientBoostingRegressor

In [6]:
from sklearn.ensemble import GradientBoostingRegressor
#
gbr = GradientBoostingRegressor(loss="ls",
                                 n_estimators=20,
                                 learning_rate=0.1,
                                 max_depth=5,
                                 subsample=0.75,
                                 verbose=1)


In [7]:
gbr.fit(xtrain,ytrain)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.0246           0.0045            4.32m
         2           0.0210           0.0036            4.23m
         3           0.0181           0.0030            4.02m
         4           0.0156           0.0025            3.89m
         5           0.0135           0.0021            3.56m
         6           0.0118           0.0018            3.27m
         7           0.0104           0.0014            3.01m
         8           0.0091           0.0012            2.77m
         9           0.0081           0.0010            2.52m
        10           0.0073           0.0008            2.31m
        20           0.0038           0.0002            0.00s


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=5, max_features=None, max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=20, presort='auto',
             random_state=None, subsample=0.75, verbose=1,
             warm_start=False)

In [8]:
from sklearn.metrics import roc_auc_score, log_loss
gbr_preds = gbr.score(xtest,ytest)
gbr_preds

0.87110408174148779

In [13]:
from sklearn.metrics import mean_squared_error
mse = (mean_squared_error(ytest, gbr.predict(xtest)))**0,5
#print("RMSE: %.4f" % mse)

In [14]:
#TODO : see wich variables provide the more informations...see below

## 6. Forêts Aléatoires

In [20]:
import multiprocessing
num_cores = multiprocessing.cpu_count()
from sklearn.cross_validation import KFold
kf = list(KFold(len(xtrain),num_cores))

from joblib import Parallel, delayed  

from sklearn.ensemble import RandomForestClassifier

def fit_model(traini):
    rfc = RandomForestClassifier()
    rfc.fit(xtrain[traini], ytrain[traini])
    return(rfc)

kf = list(KFold(len(xtrain),num_cores))
rfcs = Parallel(n_jobs=num_cores)( delayed(fit_model)(traini) for traini,_ in kf)
preds = np.empty_like(ytrain)
#for i,(_,testi) in enumerate(kf):
#    preds[testi] = rfcs[i].predict_proba(xtrain[testi])[:,1]
#print("auc:      {}".format(roc_auc_score(ytrain, preds)))
#print("log_loss: {}".format(log_loss(ytrain,preds)))

ImportError: No module named 'joblib'

In [ ]:
import random
def kfolds(k,N,seed=None):
    random.seed(seed)
    out = [ list() for _ in range(k) ]
    for n in range(N): out[random.randrange(k)].append(n)
    return(out)

K = kfolds(seed=42,k=5,N=len(train_new))
fold1 = sum(K[1:5],[]) # K[1] + K[2] + K[3] + K[4]
#fold1 = K[1]+K[2]+K[3]+K[4]
xtrain = train_new.iloc[fold1][train_new.keys()[1:]]
ytrain = train_new.iloc[fold1][train_new.keys()[0]]

xtest = train_new.iloc[K[0]][train_new.keys()[1:]]
ytest = train_new.iloc[K[0]][train_new.keys()[0]]

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()
rfr.fit(xtrain, ytrain)
    
#preds = np.empty_like(ytrain)
#for i,(_,testi) in enumerate(kf):
#    preds[testi] = rfcs[i].predict_proba(xtrain[testi])[:,1]
#print("auc:      {}".format(roc_auc_score(ytrain, preds)))
#print("log_loss: {}".format(log_loss(ytrain,preds)))

import multiprocessing
num_cores = multiprocessing.cpu_count()

from sklearn.cross_validation import KFold
kf = list(KFold(len(xtrain),num_cores))

from joblib import Parallel, delayed  

def fit_model(traini):
    rfr = RandomForestRegressor()
    rfr.fit(xtrain[traini], ytrain[traini])
    return(rfc)

kf = list(KFold(len(xtrain),num_cores))
rfcs = Parallel(n_jobs=num_cores)( delayed(fit_model)(traini) for traini,_ in kf)
preds = np.empty_like(ytrain)
#for i,(_,testi) in enumerate(kf):
#    preds[testi] = rfcs[i].predict_proba(xtrain[testi])[:,1]
#print("auc:      {}".format(roc_auc_score(ytrain, preds)))
#print("log_loss: {}".format(log_loss(ytrain,preds)))

In [ ]:
rfr.score(xtest,ytest)

In [19]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(ytest, rfr.predict(xtest))

In [22]:
mse = mse**0.5
mse

0.22889949345325988

# Cross validation Forets Aleatoires

In [23]:
import multiprocessing
num_cores = multiprocessing.cpu_count()

from sklearn.cross_validation import KFold
kf = list(KFold(len(xtrain),num_cores))

from joblib import Parallel, delayed  

def fit_model(traini):
    print('Fitting')
    rfr = RandomForestRegressor()
    rfr.fit(xtrain[traini], ytrain[traini])
    return(rfc)

kf = list(KFold(len(xtrain),num_cores))
rfcs = Parallel(n_jobs=num_cores)( delayed(fit_model)(traini) for traini,_ in kf)
preds = np.empty_like(ytrain)
#for i,(_,testi) in enumerate(kf):
#    preds[testi] = rfcs[i].predict_proba(xtrain[testi])[:,1]
#print("auc:      {}".format(roc_auc_score(ytrain, preds)))
#print("log_loss: {}".format(log_loss(ytrain,preds)))

/Applications/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Fitting
Fitting
Fitting
Fitting


JoblibIndexError: JoblibIndexError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Applications/anaconda/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...b/python3.5/site-packages/ipykernel/__main__.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

...........................................................................
/Applications/anaconda/lib/py

On voit que les résultats sont moins bons que pour les Boosted Models avec les paramètres par défaut. On cherche donc à voir s'il existe des paramètres qui donnent une meilleure AUC. Ici on utilise `GridSearchCV`, `cross_val_score` et le paramètre `n_jobs` qui sont des outils très pratiques de `scikit-learn`. En réalité `n_jobs` fait appel à `joblib` et on aurait pu l'utiliser pour les GBM également, tout comme `GridSearchCV` et `cross_val_score`. L'idée étant d'aborder les problèmes et les outils sous différents angles pour mieux les cerner.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
#Uncomment the following for GridSearch: can take 1+ hour
#from sklearn.grid_search import GridSearchCV
#gs = GridSearchCV(RandomForestClassifier(n_jobs=-1), 
#                  {'max_features' : range(1,11), 'n_estimators': [500]}, 
#                  scoring='roc_auc')
#gs.fit(xtrain,ytrain)
#print(gs.best_params_) #Outputs {'max_features':1, 'n_estimators' : 500}
cross_val_score(RandomForestClassifier(max_features=1,n_estimators=500,n_jobs=-1),
                xtrain, ytrain, cv=num_cores, scoring='roc_auc').mean()

On peut, comme pour les GBM, utiliser le modèle appris pour classer les variables par ordre d'importance.

In [ ]:
rfc = RandomForestClassifier(n_estimators=500,max_features=1,n_jobs=-1)
rfc.fit(xtrain,ytrain)
indices = np.argsort(rfc.feature_importances_)
# plot as bar chart
plt.barh(np.arange(len(features_names)), rfc.feature_importances_[indices])
plt.yticks(np.arange(len(features_names)) + 0.25, np.array(features_names)[indices])
_ = plt.xlabel('Relative importance')

On constate que c'est le même ordre d'importance que pour le GBM estimé plus haut.

## 11. Mélange

In [ ]:
def fit_gbm(traini):
    gbm = GradientBoostingClassifier(loss="deviance",
                                     n_estimators = 100,
                                     learning_rate = 0.1,
                                     max_depth = 5,
                                     subsample = .75,
                                     verbose = 0)
    gbm.fit(xtrain[traini],ytrain[traini])
    return(gbm)

gbms = Parallel(n_jobs=num_cores)( delayed(fit_gbm)(traini) for traini,_ in kf)

def fit_rfc(traini):
    rfc = RandomForestClassifier(n_estimators=500,max_features=1)
    rfc.fit(xtrain[traini],ytrain[traini])
    return(rfc)

rfcs = Parallel(n_jobs=num_cores)( delayed(fit_rfc)(traini) for traini,_ in kf)

In [ ]:
preds = np.empty_like(ytrain)
for i,(_,testi) in enumerate(kf):
    preds[testi] = (.07*rfcs[i].predict_proba(xtrain[testi])[:,1] +
                    .93*gbms[i].predict_proba(xtrain[testi])[:,1])
print("auc:      {}".format(roc_auc_score(ytrain, preds)))
print("log_loss: {}".format(log_loss(ytrain,preds)))

## 12. Prédictions sur les données de test en csv

In [ ]:
gbm = GradientBoostingClassifier(loss="deviance",
                                 n_estimators = 100,
                                 learning_rate = 0.1,
                                 max_depth = 5,
                                 subsample = .75,
                                 verbose = 0)
gbm.fit(xtrain,ytrain)
rfc = RandomForestClassifier(n_estimators=500,
                             max_features=1)
rfc.fit(xtrain,ytrain)

testname = "cs-test.csv"
testf = datapath + testname
test = pd.read_csv(testf,index_col=0)
test = imp.fit_transform(test)

preds = .07*rfc.predict_proba(test)[:,1] + .93*gbm.predict_proba(test)[:,1]

On peut vouloir stocker les prédictions sur disque sous forme de fichier csv

In [ ]:
np.savetxt('preds.csv', preds, delimiter =',')

## Use external data

In [153]:
datapath = "../challenge_28_data/"
trainf = "data2.csv"
dataf = datapath + trainf
train = pd.read_csv(dataf,';')
features_names = list(train)[1:]
train

,Finess,Raison sociale,Provenance des patients (département),Domaines d activités,"âge (deux classes >75 ans, <= 75 ans)",Nombre de séjours/séances MCO des patients en ALD,Nombre total de séjours/séances,annee,cible1
0,10007300,CLINIQUE AMBULATOIRE CENDANEG,01-Ain,D01-Digestif,<=75 ans,19,1061,2008,0.010624
1,10007300,CLINIQUE AMBULATOIRE CENDANEG,01-Ain,D01-Digestif,>75 ans,5,48,2008,0.053320
2,10007300,CLINIQUE AMBULATOIRE CENDANEG,01-Ain,D07-Cardio-vasculaire (hors cathétérismes vasc...,<=75 ans,0,1,2008,0.000000
3,10007300,CLINIQUE AMBULATOIRE CENDANEG,01-Ain,D16-Hématologie,<=75 ans,0,17,2008,0.000000
4,10007300,CLINIQUE AMBULATOIRE CENDANEG,01-Ain,D16-Hématologie,>75 ans,0,2,2008,0.000000
5,10007300,CLINIQUE AMBULATOIRE CENDANEG,01-Ain,D19-Endocrinologie,<=75 ans,0,1,2008,0.000000
6,10007300,CLINIQUE AMBULATOIRE CENDANEG,01-Ain,"D23-Toxicologie, Intoxications, Alcool",<=75 ans,0,1,2008,0.000000
7,10007300,CLINIQUE AMBULATOIRE CENDANEG,01-Ain,"D26-Activités inter spécialités, suivi thérape...",<=75 ans,2,25,2008,0.027969
8,10007300,CLINIQUE AMBULATOIRE CENDANEG,12-Aveyron,D01-Digestif,<=75 ans,0,1,2008,0.000000
9,10007300,CLINIQUE AMBULATOIRE CENDANEG,13-Bouches-du-Rhône,D01-Digestif,>75 ans,0,1,2008,0.000000


In [55]:
traina08 = "hospidiag_opendata_2008.xlsx"
train08 = pd.read_excel(datapath+traina08,sheetname = 2)

traina09 = "hospidiag_opendata_2009.xlsx"
train09 = pd.read_excel(datapath+traina09,sheetname = 2)

traina10 = "hospidiag_opendata_2010.xlsx"
train10 = pd.read_excel(datapath+traina10,sheetname = 2)

traina11 = "hospidiag_opendata_2011.xlsx"
train11 = pd.read_excel(datapath+traina11,sheetname = 2)

traina12 = "hospidiag_opendata_2012.xlsx"
train12 = pd.read_excel(datapath+traina12,sheetname = 2)

traina13 = "hospidiag_opendata_2013.xlsx"
train13 = pd.read_excel(datapath+traina13,sheetname = 2)

traina14 = "hospidiag_opendata_2014.xlsx"
train14 = pd.read_excel(datapath+traina14,sheetname = 2)

traina15 = "hospidiag_opendata_2015.xlsx"
train15 = pd.read_excel(datapath+traina15,sheetname = 2)




In [54]:
train08

,finess,rs,champ_pmsi,taa,cat,taille_MCO,taille_M,taille_C,taille_O,A7,...,CI_RH7,CI_RH8,CI_RH9,CI_RH10,CI_RH11,CI_DF1,CI_DF2,CI_DF3,CI_DF4,CI_DF5
0,010007300,CLINIQUE AMBULATOIRE CENDANEG,OQN,TAA,CLI,T1,M1,C1,NaN,.c,...,"1,38","2,28",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,010007987,CH HAUTEVILLE-LOMPNES,DGF,TAA,CH,T1,M1,C0,NaN,.c,...,"44,6","285,51","4,22","7,43","74,36",NaN,NaN,NaN,NaN,NaN
2,010008407,CH DU HAUT BUGEY,DGF,TAA,CH,T2,M2,C2,O2,.c,...,"50,5","369,28",2,"20,75","43,3",NaN,NaN,NaN,NaN,NaN
3,010780054,CH BOURG EN BRESSE,DGF,TAA,CH,T3,M4,C3,O4,.c,...,"229,73","1170,29","8,07","99,52","179,22",NaN,NaN,NaN,NaN,NaN
4,010780062,CH DE BELLEY,DGF,TAA,CH,T2,M2,C2,O2,.c,...,"50,13","319,62","0,86","22,96",37,NaN,NaN,NaN,NaN,NaN
5,010780096,CH MONTPENSIER TREVOUX,DGF,TAA,CH,T1,M1,C1,NaN,.c,...,"19,48","173,34","1,86","4,32","19,86",NaN,NaN,NaN,NaN,NaN
6,010780195,CLINIQUE CONVERT BOURG-EN-B.,OQN,TAA,CLI,T3,M2,C3,NaN,.c,...,"40,03",196,NaN,"3,8","20,84",NaN,NaN,NaN,NaN,NaN
7,010780203,CLINIQUE MUTUALISTE - AMBERIEU-EN-B.,OQN,TAA,CLI,T2,M2,C2,O3,.c,...,"31,24","114,05","0,5","2,46","3,08",NaN,NaN,NaN,NaN,NaN
8,010780294,CENTRE MÉDICAL DE DIALYSE REGINA,OQN,TAA,CLI,T0,M0,NaN,NaN,.c,...,4,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,020000022,CH DE GUISE,DGF,TAA,CH,T1,M1,C1,NaN,.c,...,"20,93","184,76","1,86","5,21","19,86",NaN,NaN,NaN,NaN,NaN


In [154]:
#Create one more DF with the all the data that we will concatenate
#train08.query(['finess'][0] == '010007300')
train08.dtypes

finess         object
rs             object
champ_pmsi     object
taa            object
cat            object
taille_MCO     object
taille_M       object
taille_C       object
taille_O       object
A7             object
A8             object
A9             object
A10            object
A11            object
A12            object
A13            object
A14            object
A15            object
F1_D           object
F1_O           object
F2_D           object
F2_O           object
F3_D           object
F3_O           object
F4_D           object
F4_O           object
F5_D           object
F5_O           object
F6_D           object
F6_O           object
               ...   
CI_F11_D      float64
CI_F11_O      float64
CI_F12_D      float64
CI_F12_O      float64
CI_F13_D      float64
CI_F13_O      float64
CI_F14_D      float64
CI_F14_O      float64
CI_F15_D      float64
CI_F15_O      float64
CI_F16_D      float64
CI_F16_O      float64
CI_F17_D      float64
CI_F17_O       object
CI_RH1    

In [125]:
more_information = pd.DataFrame(columns = (list(train08[1:])))
more_information

,finess,rs,champ_pmsi,taa,cat,taille_MCO,taille_M,taille_C,taille_O,A7,...,CI_RH7,CI_RH8,CI_RH9,CI_RH10,CI_RH11,CI_DF1,CI_DF2,CI_DF3,CI_DF4,CI_DF5


In [126]:

for n in range(20,25):
    #train.count à la place de 2
    
    print("n: %i"% n)
    annee = train['annee'][n]
    print("annee: %i"% annee)
    finess = train['Finess'][n]
    print("finess: %i"% finess)
    
    if annee == 2008:
        more_information = more_information.append(train08.loc[train08['finess'] == train['Finess'][n]])
    else:
        more_information = more_information.append(train08.loc[train08['finess'] == train['Finess'][n]])

more_information
        

n: 20
annee: 2008
finess: 10007300
n: 21
annee: 2008
finess: 10007300
n: 22
annee: 2008
finess: 10007300
n: 23
annee: 2008
finess: 10007300
n: 24
annee: 2008
finess: 10007987


,finess,rs,champ_pmsi,taa,cat,taille_MCO,taille_M,taille_C,taille_O,A7,...,CI_RH7,CI_RH8,CI_RH9,CI_RH10,CI_RH11,CI_DF1,CI_DF2,CI_DF3,CI_DF4,CI_DF5


In [149]:
train['Finess'] = train['Finess'].astype(str)
#str(train['Finess'])
#train['Finess'][0] == train08['finess'][0]

In [151]:
train['Finess'][0]

'10007300'

In [141]:
more_information = train08.loc[train08['finess'] == '010007300']
more_information = more_information.append(train08.loc[train08['finess'] == str(train['Finess'][0])])
more_information

,finess,rs,champ_pmsi,taa,cat,taille_MCO,taille_M,taille_C,taille_O,A7,...,CI_RH7,CI_RH8,CI_RH9,CI_RH10,CI_RH11,CI_DF1,CI_DF2,CI_DF3,CI_DF4,CI_DF5
0,010007300,CLINIQUE AMBULATOIRE CENDANEG,OQN,TAA,CLI,T1,M1,C1,NaN,.c,...,"1,38","2,28",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
#Age to numeric

train['âge (deux classes >75 ans, <= 75 ans)'][train['âge (deux classes >75 ans, <= 75 ans)']=='<=75 ans']=0
train['âge (deux classes >75 ans, <= 75 ans)'][train['âge (deux classes >75 ans, <= 75 ans)']=='>75 ans']=1
train['âge (deux classes >75 ans, <= 75 ans)'] = pd.to_numeric(train['âge (deux classes >75 ans, <= 75 ans)'])

#Raison sociale / Labelencoder

le.fit(train['Raison sociale'])
list(le.classes_)
train['Raison sociale'] = le.transform(train['Raison sociale'])

#pd.get_dummies
train_DA = pd.get_dummies(train['Domaines d activités'])
train_PP = pd.get_dummies(train['Provenance des patients (département)'])
train_new = pd.concat([train,train_DA,train_PP],axis=1)

#Suppression des doublons

del train_new['Domaines d activités']
del train_new['Provenance des patients (département)']
del train['Finess']
#Finess <=> Raison sociale - We choose here to learn as well using the hospitals name 
#(team/equipments,etc...)

#Reorder the column
cols = train_new.columns.tolist()
cols =  cols[5:] + cols[1:5]
cols
train_new=train_new[cols]

KeyboardInterrupt: 